In [1]:
!sudo apr-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh
from IPython.display import clear_output

####
import os
import threading
import subprocess
import requests
import json

def ollama():
  os.environ['OLLAMA_HOST']='0.0.0.0:11434'
  os.environ['OLLAMA_ORIGINS']='*'
  subprocess.Popen(["ollama","serve"])

ollama_thread=threading.Thread(target=ollama)
ollama_thread.start()

sudo: apr-get: command not found
>>> Downloading ollama...
######################################################################## 100.0%#=#=-#  #                                                                                                               34.3%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Couldn't find '/home/codespace/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAILgD467nEBdYt+NKYtMm1G7kuCu09g5pRdcjw9aCAjPE



2024/07/31 02:16:40 routes.go:1099: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/codespace/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-07-31T02:16:40.563Z level=INFO source=images.go:784 msg="total blobs: 0"
time=2024-07-31T02:16:40.563Z level=INFO source=images.go:791 msg="total unused blobs removed: 0"
time=2024-07-31

In [2]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [3]:
!pip install -U lightrag[ollama]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 671.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 37.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 21.3 MB/s eta 0:00:0000:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient,GroqAPIClient

import time
qa_template = r"""<SYS>
you are a helpful assistant.
</Sys>
user: {{input_str}}
You:"""

class SimpleQA(Component):
  def __init__(self,model_client:ModelClient, model_kwargs:dict):
    super().__init__()
    self.generator = Generator(
        model_client=model_client,
        model_kwargs=model_kwargs,
        template=qa_template,
    )
  def call(self,input:dict)->str:
    return self.generator.call({"input_str":str(input)})
  async def acall(self,input:dict)->str:
    return await self.generator.acall({"input_str":str(input)})
   

In [5]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model={
    "model_client":OllamaClient(),
    "model_kwargs":{"model":"llama3.1:8b"}
}
qa=SimpleQA(**model)
output=qa("what is happiness")
display(Markdown(f"**Answer:** {output.data}"))

time=2024-07-31T02:20:15.265Z level=INFO source=memory.go:309 msg="offload to cpu" layers.requested=-1 layers.model=33 layers.offload=0 layers.split="" memory.available="[13.3 GiB]" memory.required.full="5.8 GiB" memory.required.partial="0 B" memory.required.kv="1.0 GiB" memory.required.allocations="[5.8 GiB]" memory.weights.total="4.7 GiB" memory.weights.repeating="4.3 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="560.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-07-31T02:20:15.266Z level=INFO source=server.go:383 msg="starting llama server" cmd="/tmp/ollama2541654620/runners/cpu_avx2/ollama_llama_server --model /home/codespace/.ollama/models/blobs/sha256-87048bcd55216712ef14c11c2c303728463207b165bf18440b9b84b07ec00f87 --ctx-size 8192 --batch-size 512 --embedding --log-disable --no-mmap --parallel 4 --port 34053"
time=2024-07-31T02:20:15.266Z level=INFO source=sched.go:437 msg="loaded runners" count=1
time=2024-07-31T02:20:15.266Z level=INFO source=server.go:583 

INFO [main] build info | build=1 commit="d94c6e0" tid="126785078486912" timestamp=1722392415
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 0 | " tid="126785078486912" timestamp=1722392415 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="34053" tid="126785078486912" timestamp=1722392415


time=2024-07-31T02:20:15.517Z level=INFO source=server.go:617 msg="waiting for server to become available" status="llm server loading model"
llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.7999 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_meta: n_merges         = 280147
llm_load_print_meta: vocab_only       = 0
llm_load_print_meta: n_ctx_train      = 131072
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_swa            = 0
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     

INFO [main] model loaded | tid="126785078486912" timestamp=1722392429


time=2024-07-31T02:20:29.818Z level=INFO source=server.go:622 msg="llama runner started in 14.55 seconds"


[GIN] 2024/07/31 - 02:21:40 | 200 |         1m25s |       127.0.0.1 | POST     "/api/generate"


**Answer:** What a wonderful topic!

Happiness can be defined in many ways, but at its core, it's a positive emotional state that brings joy and fulfillment to one's life. It's the feeling of being content, satisfied, and even blissful.

Research has identified various factors that contribute to happiness, such as:

1. **Positive relationships**: Strong connections with loved ones, friends, and community.
2. **Gratitude**: Practicing appreciation for the good things in life.
3. **Physical health**: Engaging in regular exercise, healthy eating, and adequate sleep.
4. **Meaningful activities**: Pursuing hobbies, passions, and work that bring purpose and fulfillment.
5. **Resilience**: Developing coping skills to navigate life's challenges with optimism and adaptability.

According to the Harvard Happiness Study (1938-2012), people who reported being happy were more likely to:

* Live longer
* Have stronger immune systems
* Experience better mental health
* Have healthier relationships
* Enjoy greater overall well-being

While happiness can vary from person to person, it's clear that cultivating a positive mindset and focusing on the things that bring joy and fulfillment are essential for achieving true happiness.

Would you like me to explore any specific aspect of happiness further?

In [72]:
# !pip install flask
# %pip install jsonify
# import flast module
from flask import Flask, render_template,jsonify,request

# instance of flask application
app = Flask(__name__)
 
# home route that returns below text when root url is accessed
@app.route("/")
def home():
   return render_template("index.html")

@app.route("/llm_run",methods=["POST","GET"])
def llm_run():
    prompt=None
    print(prompt,request.method)
    if request.method =="POST":
       print("Value",request.form)
       prompt= request.form["search"]
       print(str(prompt))
       output=qa(prompt)
       print("running....")
    # display(Markdown(f"**Answer:** {output.data}")) str(output.data)
    return jsonify({'htmlresponse': render_template('response.html',resp=output)})
 
if __name__ == '__main__':  
   app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/Jul/2024 05:43:42] "GET / HTTP/1.1" 200 -


In [7]:
output=qa("what is 2+2")
clear_output()
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** The answer to 2 + 2 is 4. Is there anything else I can help you with?